In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_pickle('data.pandas')

In [3]:
data.dtypes

address                            object
body                               object
date_sent                  datetime64[ns]
readable_date              datetime64[ns]
service_center                     object
read                                int64
type                                int64
id                                float64
ClusterId                         float64
DistrictCity                       object
Jamoat                             object
DateCollectionStart                object
MONEY                             float64
DATE                               object
COUNT                             float64
PSU_ID                            float64
URB_RUR                            object
REGION                             object
DISTRICT                           object
LONG                              float64
LAT                               float64
dateCollectionStartDays            object
_merge                           category
dtype: object

In [4]:
data['dateSentDays'] = data['date_sent'].dt.date
data['dateCollectionStartDays'] = pd.to_datetime(data['DateCollectionStart'], infer_datetime_format=True).dt.date
data = data[data['dateSentDays'] >= data['dateCollectionStartDays']]

In [5]:
data.date_sent.describe()

count                  129622
unique                 117421
top       2017-01-15 19:22:50
freq                       15
first     2016-10-19 17:28:36
last      2017-03-23 16:56:32
Name: date_sent, dtype: object

In [6]:
timeStampDiff = data['readable_date']- data['date_sent']
timeStampDiff.describe()

count                    129622
mean     0 days 00:47:14.937549
std      0 days 03:35:44.991749
min             0 days 00:00:00
25%             0 days 00:00:04
50%             0 days 00:00:08
75%             0 days 00:02:02
max             1 days 22:38:48
dtype: object

In [57]:
# Still after eliminating records with date_sent = 0, there are differences of almost two days

In [7]:
print('Number of messages with at least one hour diff: ', timeStampDiff[timeStampDiff > pd.to_timedelta(1, 'h')].count())
print('Number of messages with at least one day diff: ', timeStampDiff[timeStampDiff > pd.to_timedelta(1, 'D')].count())

Number of messages with at least one hour diff:  10249
Number of messages with at least one day diff:  525


In [59]:
# This will change to a loop over boxes
box = data[['id', 'date_sent', 'body']][data['id'] == 1301]
box.count()

id           445
date_sent    445
body         445
dtype: int64

In [60]:
start = box['date_sent'].min()
end = box['date_sent'].max()
fullHour = pd.DatetimeIndex(freq='H', start=start, end=end + pd.to_timedelta(1, 'H'), closed=None).floor('H')

In [61]:
box.set_index('date_sent', drop=True, inplace=True)
box.index = box.index.floor('H') # But doing this can create two records in the same hour
box.reset_index(inplace=True)

In [62]:
boxNoDup = pd.DataFrame(box[['id', 'date_sent']].groupby('date_sent').count(), copy=True)
boxNoDup = boxNoDup.reindex(fullHour, fill_value=0)

In [63]:
boxNoDup['countDayLeading'] = boxNoDup['id'].rolling(window=24, min_periods=1).sum()
boxNoDup['countDayLeading'] = boxNoDup['countDayLeading'] - boxNoDup['id'] # to avoid double counting that hour

boxNoDup['countDayTrailing'] = boxNoDup['countDayLeading'].tshift(-23)
boxNoDup.loc[(boxNoDup.index.max() - pd.to_timedelta(23, 'H')):, 'countDayTrailing'] = \
    [sum(boxNoDup['id'].loc[boxNoDup.index.max() - pd.to_timedelta(23 - i, 'H'):]) for i in range(24)]

boxNoDup['missingHour'] = (boxNoDup['countDayTrailing'] + (boxNoDup['countDayLeading']) < 2)

In [64]:
boxNoDup['missingHour'].sum() / boxNoDup.shape[0]

0.2068111455108359

In [8]:
def countMissingHours(boxId=1301, hrsAround=24, data=data):
    
    box = data[['id', 'date_sent', 'body']][data['id'] == boxId]
    
    try:
        
        start = box['date_sent'].min()
        end = box['date_sent'].max()
        fullHour = pd.DatetimeIndex(freq='H', start=start, end=end + pd.to_timedelta(1, 'H'), closed=None).floor('H')

        box.set_index('date_sent', drop=True, inplace=True)
        box.index = box.index.floor('H') # But doing this can create two records in the same hour
        box.reset_index(inplace=True)

        boxNoDup = pd.DataFrame(box[['id', 'date_sent']].groupby('date_sent').count(), copy=True)
        boxNoDup = boxNoDup.reindex(fullHour, fill_value=0)

        boxNoDup['countDayLeading'] = boxNoDup['id'].rolling(window=hrsAround, min_periods=1).sum()
        boxNoDup['countDayLeading'] = boxNoDup['countDayLeading'] - boxNoDup['id'] # to avoid double counting that hour

        boxNoDup['countDayTrailing'] = boxNoDup['countDayLeading'].tshift(-(hrsAround - 1))
        boxNoDup.loc[(boxNoDup.index.max() - pd.to_timedelta((hrsAround - 1), 'H')):, 'countDayTrailing'] = \
            [sum(boxNoDup['id'].loc[boxNoDup.index.max() - pd.to_timedelta((hrsAround - 1) - i, 'H'):]) for i in range(hrsAround)]

        boxNoDup['isMissing'] = (boxNoDup['countDayTrailing'] + (boxNoDup['countDayLeading']) < 2)
        boxNoDup.reset_index(inplace=True)
        boxNoDup.rename(columns={'index':'date_sent'}, inplace=True)
        boxNoDup['id'] = boxId
        return boxNoDup[['date_sent', 'id', 'isMissing']]
    
    except ValueError:
        #pass
        print('Error in box ', boxId)
    

In [10]:
boxList = set(data['id'])
missingHours = pd.DataFrame(columns=['date_sent', 'id', 'isMissing'])
for box in boxList:
    missingHours = pd.concat([missingHours, countMissingHours(boxId=box)], ignore_index=True)

Error in box  1039.0
Error in box  1124.0
Error in box  1135.0


In [11]:
len(missingHours['id'].value_counts())

277

In [13]:
data[(data['id'] == 1039) | (data['id'] == 1124) | (data['id'] == 1135)]

,address,body,date_sent,readable_date,service_center,read,type,id,ClusterId,DistrictCity,...,COUNT,PSU_ID,URB_RUR,REGION,DISTRICT,LONG,LAT,dateCollectionStartDays,_merge,dateSentDays
22405,+992900004351,Power Failure is running id 1135 | Monitoring...,2016-11-25 13:10:38,2016-11-25 13:10:38,+992907777777,0,1,1135.0,48.0,Jirgatol,...,69.0,48.0,Rural,DRS,Jirgital,71.908341,39.201393,2016-11-23,both,2016-11-25
22406,+992900004351,Power Failure is running id 1135 | Monitoring...,2016-11-25 13:10:42,2016-11-25 13:10:42,+992907777777,0,1,1135.0,48.0,Jirgatol,...,69.0,48.0,Rural,DRS,Jirgital,71.908341,39.201393,2016-11-23,both,2016-11-25
22407,+992900004351,Power Failure detected. id 1135,2016-11-25 13:24:48,2016-11-25 13:24:48,+992907777777,0,1,1135.0,48.0,Jirgatol,...,69.0,48.0,Rural,DRS,Jirgital,71.908341,39.201393,2016-11-23,both,2016-11-25
22408,+992900004351,Power Back detected. id 1135,2016-11-25 13:25:08,2016-11-25 13:25:08,+992907777777,0,1,1135.0,48.0,Jirgatol,...,69.0,48.0,Rural,DRS,Jirgital,71.908341,39.201393,2016-11-23,both,2016-11-25
57983,+992935710968,Power Failure is running ID 1039 | Monitoring...,2016-11-11 05:40:51,2016-11-11 05:40:51,+992939999999,0,1,1039.0,98.0,Khovaling,...,24.0,98.0,Rural,Khatlon,Khovaling,69.933057,38.329808,2016-11-11,both,2016-11-11
75812,+992935710423,Test SMS from Power Failure Monitor:Power Fail...,2017-01-25 15:58:38,2017-01-25 15:58:49,+992939999999,0,1,1124.0,137.0,Isfara,...,24.0,137.0,Rural,Sugd,Isfara,70.579609,40.143452,2016-11-21,both,2017-01-25
75813,+992935710423,Test SMS from Power Failure Monitor:Power Back...,2017-01-25 15:58:42,2017-01-25 15:59:13,+992939999999,0,1,1124.0,137.0,Isfara,...,24.0,137.0,Rural,Sugd,Isfara,70.579609,40.143452,2016-11-21,both,2017-01-25


In [15]:
workDayMF = pd.DatetimeIndex( missingHours['date_sent']).dayofweek < 6 # Mon=0 - Fri=5
workDayS = pd.DatetimeIndex( missingHours['date_sent']).dayofweek == 6 # Saturday (private sector may work)
for holiday in ['2016-11-06',
               '2016-11-09',
               '2017-01-02', 
               '2017-03-08', 
               '2017-03-21', '2017-03-22', '2017-03-23', '2017-03-24',
               '2017-05-01',
               '2017-05-09',
               '2017-06-25',
               '2017-06-26',
               '2017-06-27',
               '2017-09-01',
               '2017-09-11']:
    # Idi Qurbon changes with year. Sep 1st this year
    # Indenependece day is Sep 9, but that was Sat in 2017, moved to Mon (sep 11)
    # New years day is 1/1 but tht was Sun in 2017, moved to 01-02
    # Ramadan canges year to year. In 2017, on June Sun 25, jun Mon 26
    workDayMF[missingHours['date_sent'].dt.date == pd.to_datetime(holiday).date()] = False
    workDayS[missingHours['date_sent'].dt.date == pd.to_datetime(holiday).date()] = False


In [16]:
workDayMF[ (pd.DatetimeIndex(missingHours['date_sent']).hour < 9) | 
          (pd.DatetimeIndex(missingHours['date_sent']).hour > 17) ] = False
workDayS[ (pd.DatetimeIndex(missingHours['date_sent']).hour < 9) | 
          (pd.DatetimeIndex(missingHours['date_sent']).hour > 17) ] = False

In [17]:
missingHours['month'] = pd.DatetimeIndex(missingHours['date_sent']).month
missingHours['workDayMF'] = workDayMF
missingHours['workDayS'] = workDayS

In [18]:
missingHours.to_pickle('missingHours.pandas')